### Project: Domain Adaptation of Portuguese SLMs via Self-Supervised Fine-Tuning with LoRA
MO436C - Introduction to Self-Supervised Learning (SSRL)

**Team Members:**
- Alejandro Núñez Arroyo. <a href="mailto:a299215@dac.unicamp.br">a299215@dac.unicamp.br</a>  
- Daniel da Costa Nunes Resende Neto. <a href="mailto:d169408@dac.unicamp.br">d169408@dac.unicamp.br</a>  
- José Augusto de Almeida Neto. <a href="mailto:j299218@dac.unicamp.br">j299218@dac.unicamp.br</a>  

*Instituto de Computação (IC), Universidade Estadual de Campinas (UNICAMP)*  
*Campinas, November 2025*

---

#### License

This notebook and its source code are released under the **GNU General Public License v3.0 (GPLv3)**.  
You are free to use, modify, and redistribute this work under the following terms:

> **GNU General Public License v3.0**  
> Copyright © 2025 The Authors listed above  
>
> This program is free software: you can redistribute it and/or modify  
> it under the terms of the GNU General Public License as published by  
> the Free Software Foundation, either version 3 of the License, or  
> (at your option) any later version.  
>
> This program is distributed in the hope that it will be useful,  
> but **without any warranty**; without even the implied warranty of  
> merchantability or fitness for a particular purpose. See the  
> GNU General Public License for more details.  
>
> You should have received a copy of the GNU General Public License  
> along with this program. If not, see  
> [https://www.gnu.org/licenses/gpl-3.0.en.html](https://www.gnu.org/licenses/gpl-3.0.en.html).

---

# Notebook 1: Data

This notebook focuses on the data **acquisition, exploration, and preprocessing** stage of the project.  
It is responsible for **collecting, cleaning, and organizing datasets** that will later be used for modeling and experimentation.

---

**Overview**

The main objectives of this notebook are:

1. **Setup & Imports**  
   Load all core Python libraries (NumPy, Pandas, PyTorch, etc.) and configure the environment.

2. **Load and Analyze the MMLU Dataset**  
   - Import the **Portuguese (PT-BR)** version of the *Massive Multitask Language Understanding* (MMLU) dataset from Hugging Face.
   - Map each subject area to a higher-level **macrodomain** (e.g., Law, Medicine, Economics).
   - Generate statistics, verify mapping coverage, and perform **train/test splits** by macrodomain.

3. **Load the Wikipedia PT-BR Dataset**  
   - Download and concatenate multiple `.parquet` files containing Wikipedia articles in Portuguese.
   - Use **keyword-based filtering** and **semantic embedding similarity** to select articles relevant to Law, Governance, and Ethics.
   - Apply efficient filtering with multithreading for scalability.
   - Combine results from keyword and embedding approaches.
   - Remove duplicates and generate the final curated dataset `wiki_final.csv`.

**Output Artifacts**  
   - `mmlu_train.csv` / `mmlu_test.csv` — split subsets of the MMLU dataset.
   - `wiki_keyword.csv` — articles filtered by keywords.
   - `wiki_final.csv` — combined final dataset.


## Summary

* [Part 1: Setup & Imports](#1-setup--imports)
* [Part 2: Massive Multitask Language Understanding (MMLU)](#2-massive-multitask-language-understanding-mmlu)
  - [2.1 Loading the Dataset](#21-loading-the-dataset)
  - [2.2 Mapping Subjects to Macrodomains](#22-mapping-subjects-to-macrodomains)
  - [2.3 Sampling Questions by Macrodomain](#23-sampling-questions-by-macrodomain)
  - [2.4 Train/Test Split](#24-traintest-split)
* [Part 3: Wikipedia PT-BR](#3-wikipedia-pt-br)
  - [3.1 Loading the Dataset](#31-loading-the-dataset)
  - [3.2 Keyword-Based Filtering](#32-keyword-based-filtering)
  - [3.3 Semantic Filtering with Embeddings](#33-semantic-filtering-with-embeddings)
  - [3.4 Final Dataset](#34-final-dataset)

<!-- ## 1. Setup & Imports -->
## 1. Setup & Imports <a id="part_01"></a>


Here we load the main libraries that will be used throughout this notebook.

In [1]:
import glob
import os
import re
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Create output directory
output_path = "data/"
os.makedirs(output_path, exist_ok=True)

## 2. Massive Multitask Language Understanding (MMLU) <a id="part_02"></a>

This section loads and prepares the **Portuguese (PT-BR)** version of the [MMLU dataset](https://huggingface.co/datasets/openai/MMMLU), a benchmark for evaluating general knowledge and reasoning across multiple academic subjects.

**Main steps:**
1. Load the MMLU PT-BR dataset from Hugging Face.  
2. Map each subject to a broader **macrodomain** (e.g., Law, Medicine, Psychology).  
3. Generate summary statistics and verify coverage.  
4. Perform a **train/test split** stratified by subject.  
5. Save the processed files for downstream training (`mmlu_train.csv`, `mmlu_test.csv`).

> This step ensures the MMLU dataset is clean, organized, and aligned with higher-level knowledge domains.


### 2.1 Loading the Dataset

Import the **MMLU PT-BR** CSV file and inspect its structure and distribution of subjects.

In [2]:
# Load the MMLU Portuguese (Brazil) dataset
mmmlu_pt_path = "https://drive.google.com/uc?export=download&id=1WwhkiRUZaDSj-3aWm0iTujMIFn4KUGhh"
df_mmmlu_pt = pd.read_csv(mmmlu_pt_path)
df_mmmlu_pt.head()

,Unnamed: 0,Question,A,B,C,D,Answer,Subject
0,0,Encontre o ângulo para a extensão de campo dad...,0,4,2,6,B,abstract_algebra
1,1,"Considere p = (1, 2, 5, 4)(2, 3) em S_5. Encon...",8,2,24,120,C,abstract_algebra
2,2,Encontre todos os zeros no campo finito indica...,0,1,"0,1","0,4",D,abstract_algebra
3,3,Declaração 1 | Um grupo quociente de um grupo ...,"Verdadeiro, Verdadeiro","Falso, Falso","Verdadeiro, Falso","Falso, Verdadeiro",B,abstract_algebra
4,4,Encontre o produto dos polinômios dados no ane...,2x^2 + 5,6x^2 + 4x + 6,0,x^2 + 1,B,abstract_algebra


In [3]:
# Count rows for each category in the 'Subject' column
subject_counts = df_mmmlu_pt['Subject'].value_counts()
subject_counts

Subject
professional_law                       1534
moral_scenarios                         895
miscellaneous                           783
professional_psychology                 612
high_school_psychology                  545
high_school_macroeconomics              390
elementary_mathematics                  378
moral_disputes                          346
prehistory                              324
philosophy                              311
high_school_biology                     310
nutrition                               306
professional_accounting                 282
professional_medicine                   272
high_school_mathematics                 270
clinical_knowledge                      265
security_studies                        245
high_school_microeconomics              238
high_school_world_history               237
conceptual_physics                      235
marketing                               234
human_aging                             223
high_school_statistics  

### 2.2 Mapping Subjects to Macrodomains

In this step, we grouped the detailed **MMLU subjects** into broader thematic **macrodomains**  
(e.g., *Law, Governance, and Ethics*, *Medicine, Health, and Life Sciences*, *Psychology, Human Behavior, and Society*).

To ensure a coherent and well-balanced structure, the **subject–macrodomain mapping** was created with the assistance of **ChatGPT 5**,  
which analyzed the subject list and organized them into meaningful high-level categories.

---

#### Prompt used with ChatGPT 5

> “Help me group the following subjects into coherent *macrodomains*. 
> Each macrodomain should represent a broad conceptual or thematic area that can be mapped to Wikipedia categories or article collections, allowing me to gather related Wikipedia data for fine-tuning a specialized language model (SLM). 
> 
> The subjects I’m providing come from my QA dataset. Please group them in a way that maximizes semantic cohesion and domain relevance for model fine-tuning — ensuring that each macrodomain corresponds to a consistent knowledge field, minimizes overlap, and can be associated with clear, well-defined Wikipedia subdomains. 
> 
> For each macrodomain you identify, provide: 
> 
> 1. A **macrodomain name** (concise but descriptive). 
> 2. A **short definition or scope note** (what it covers and excludes). 
> 3. The **list of subjects** from my dataset that belong to it.”
>
> **Subjects:**
> ```text
> professional_law      1534
> moral_scenarios        895
> miscellaneous          783
> professional_psychology 612
> high_school_psychology  545
> ...
> ```

* The evaluated mapping was reviewed by the authors and is used throughout this notebook to ensure consistent subject categorization across all analyses and datasets.


In [4]:
# Mapping dictionary for Subject → Macrodomain
subject_to_macrodomain = {
    # --- 1. Law, Governance, and Ethics ---
    'professional_law': 'Law, Governance, and Ethics',
    'international_law': 'Law, Governance, and Ethics',
    'jurisprudence': 'Law, Governance, and Ethics',
    'business_ethics': 'Law, Governance, and Ethics',
    'moral_scenarios': 'Law, Governance, and Ethics',
    'moral_disputes': 'Law, Governance, and Ethics',
    'philosophy': 'Law, Governance, and Ethics',
    'logical_fallacies': 'Law, Governance, and Ethics',

    # --- 2. Psychology, Human Behavior, and Society ---
    'professional_psychology': 'Psychology, Human Behavior, and Society',
    'high_school_psychology': 'Psychology, Human Behavior, and Society',
    'sociology': 'Psychology, Human Behavior, and Society',
    'human_sexuality': 'Psychology, Human Behavior, and Society',
    'human_aging': 'Psychology, Human Behavior, and Society',

    # --- 3. Medicine, Health, and Life Sciences ---
    'professional_medicine': 'Medicine, Health, and Life Sciences',
    'clinical_knowledge': 'Medicine, Health, and Life Sciences',
    'college_medicine': 'Medicine, Health, and Life Sciences',
    'anatomy': 'Medicine, Health, and Life Sciences',
    'medical_genetics': 'Medicine, Health, and Life Sciences',
    'virology': 'Medicine, Health, and Life Sciences',
    'nutrition': 'Medicine, Health, and Life Sciences',
    'college_biology': 'Medicine, Health, and Life Sciences',
    'high_school_biology': 'Medicine, Health, and Life Sciences',

    # --- 4. Economics, Business, and Management ---
    'high_school_microeconomics': 'Economics, Business, and Management',
    'high_school_macroeconomics': 'Economics, Business, and Management',
    'econometrics': 'Economics, Business, and Management',
    'professional_accounting': 'Economics, Business, and Management',
    'management': 'Economics, Business, and Management',
    'marketing': 'Economics, Business, and Management',
    'public_relations': 'Economics, Business, and Management',

    # --- 5. Political Science, Security, and Global Affairs ---
    'high_school_government_and_politics': 'Political Science, Security, and Global Affairs',
    'us_foreign_policy': 'Political Science, Security, and Global Affairs',
    'security_studies': 'Political Science, Security, and Global Affairs',
    'international_law': 'Political Science, Security, and Global Affairs',  # cross-listed

    # --- 6. Natural Sciences and Engineering ---
    'conceptual_physics': 'Natural Sciences and Engineering',
    'high_school_physics': 'Natural Sciences and Engineering',
    'college_physics': 'Natural Sciences and Engineering',
    'high_school_chemistry': 'Natural Sciences and Engineering',
    'college_chemistry': 'Natural Sciences and Engineering',
    'electrical_engineering': 'Natural Sciences and Engineering',
    'astronomy': 'Natural Sciences and Engineering',

    # --- 7. Mathematics, Statistics, and Computer Science ---
    'elementary_mathematics': 'Mathematics, Statistics, and Computer Science',
    'high_school_mathematics': 'Mathematics, Statistics, and Computer Science',
    'high_school_statistics': 'Mathematics, Statistics, and Computer Science',
    'college_mathematics': 'Mathematics, Statistics, and Computer Science',
    'abstract_algebra': 'Mathematics, Statistics, and Computer Science',
    'formal_logic': 'Mathematics, Statistics, and Computer Science',
    'college_computer_science': 'Mathematics, Statistics, and Computer Science',
    'high_school_computer_science': 'Mathematics, Statistics, and Computer Science',
    'machine_learning': 'Mathematics, Statistics, and Computer Science',
    'computer_security': 'Mathematics, Statistics, and Computer Science',

    # --- 8. History, Geography, and Culture ---
    'prehistory': 'History, Geography, and Culture',
    'high_school_world_history': 'History, Geography, and Culture',
    'high_school_european_history': 'History, Geography, and Culture',
    'high_school_us_history': 'History, Geography, and Culture',
    'high_school_geography': 'History, Geography, and Culture',
    'global_facts': 'History, Geography, and Culture',

    # --- 9. Religion and Worldviews ---
    'world_religions': 'Religion and Worldviews',

    # --- 10. Miscellaneous ---
    'miscellaneous': 'Miscellaneous and Cross-domain Knowledge',
}

# Add new column to the DataFrame
df_mmmlu_pt['Macrodomain'] = df_mmmlu_pt['Subject'].map(subject_to_macrodomain)

# Check mapping coverage
unmapped = df_mmmlu_pt[df_mmmlu_pt['Macrodomain'].isna()]['Subject'].unique()
if len(unmapped) > 0:
    print("\nUnmapped subjects detected:", unmapped)
else:
    print("\nAll subjects successfully mapped to macrodomains!")

# Count rows for each Macrodomain
macrodomain_counts = df_mmmlu_pt['Macrodomain'].value_counts()
print(macrodomain_counts)

# Preview updated DataFrame
df_mmmlu_pt.head()


All subjects successfully mapped to macrodomains!
Macrodomain
Law, Governance, and Ethics                        3457
Medicine, Health, and Life Sciences                1871
Psychology, Human Behavior, and Society            1712
Mathematics, Statistics, and Computer Science      1602
Economics, Business, and Management                1471
History, Geography, and Culture                    1228
Natural Sciences and Engineering                   1088
Miscellaneous and Cross-domain Knowledge            783
Political Science, Security, and Global Affairs     659
Religion and Worldviews                             171
Name: count, dtype: int64


,Unnamed: 0,Question,A,B,C,D,Answer,Subject,Macrodomain
0,0,Encontre o ângulo para a extensão de campo dad...,0,4,2,6,B,abstract_algebra,"Mathematics, Statistics, and Computer Science"
1,1,"Considere p = (1, 2, 5, 4)(2, 3) em S_5. Encon...",8,2,24,120,C,abstract_algebra,"Mathematics, Statistics, and Computer Science"
2,2,Encontre todos os zeros no campo finito indica...,0,1,"0,1","0,4",D,abstract_algebra,"Mathematics, Statistics, and Computer Science"
3,3,Declaração 1 | Um grupo quociente de um grupo ...,"Verdadeiro, Verdadeiro","Falso, Falso","Verdadeiro, Falso","Falso, Verdadeiro",B,abstract_algebra,"Mathematics, Statistics, and Computer Science"
4,4,Encontre o produto dos polinômios dados no ane...,2x^2 + 5,6x^2 + 4x + 6,0,x^2 + 1,B,abstract_algebra,"Mathematics, Statistics, and Computer Science"


### 2.3 Sampling Questions by Macrodomain

Display a few representative questions from each **macrodomain** to:
- Visually confirm that the mapping and labeling are coherent.
- Analyze and select the most fitted macrodomain for the experiments

In [ ]:
# Print sample questions organized by Macrodomain and Subject
n_samples_per_subject = 2
macrodomain_order = (
    df_mmmlu_pt['Macrodomain']
    .value_counts()
    .index
)

for macrodomain in macrodomain_order:
    group = df_mmmlu_pt[df_mmmlu_pt['Macrodomain'] == macrodomain]

    print("=" * 100)
    print(f"📘 MACRODOMAIN: {macrodomain.upper()} - Total questions: {len(group)}")
    print("=" * 100)
    
    # Iterate over subjects inside each Macrodomain
    for subject, sub_df in group.groupby('Subject'):
        print(f"\n🔹 Subject: {subject}\n" + "-" * 80)
        
        # Take sample questions
        samples = sub_df.sample(n=min(n_samples_per_subject, len(sub_df)), random_state=42)
        
        for i, row in samples.iterrows():
            print(f"Question: {row['Question']}")
            print(f"a) {row['A']}")
            print(f"b) {row['B']}")
            print(f"c) {row['C']}")
            print(f"d) {row['D']}")
            print(f"Correct Answer: {row['Answer']}")
            print("-" * 60)
    print("\n\n")


📘 MACRODOMAIN: LAW, GOVERNANCE, AND ETHICS - Total questions: 3457

🔹 Subject: business_ethics
--------------------------------------------------------------------------------
Question: ________________ o ambiente de trabalho envolve capacitar os empregados, por exemplo, por meio de “enriquecimento do cargo”, pelo qual os empregados recebem um escopo maior para decidir como organizar seu trabalho, ou “expansão do cargo”, por onde os empregados recebem mais tarefas.
a) Revigorar
b) Renovar
c) Revitalizar
d) Reumanizar
Correct Answer: D
------------------------------------------------------------
Question: Conforme Mitchell et al (1997), ___________, a capacidade percebida de uma parte interessada de influenciar a ação organizacional, _______________, se a organização percebe as ações da parte interessada como desejáveis, apropriadas e corretas, e ____________, a imediação da atenção que a parte interessada exige, determinam a __________ da parte interessada.
a) Poder, Legitimidade, Sali

### 2.4 Train/Test Split

Split the dataset for the selected macrodomain (*Law, Governance, and Ethics*) into **training (70%)** and **test (30%)** subsets,  
stratified by subject for balanced evaluation.

In [ ]:
# Generate counts of questions per Macrodomain and Subject
subject_counts = df_mmmlu_pt.groupby(['Macrodomain', 'Subject']).size().reset_index(name='Count')
subject_counts_sorted = subject_counts.sort_values(by='Count', ascending=False)

# Select 'Law, Governance, and Ethics' Macrodomain    
subject_counts_sorted[subject_counts_sorted['Macrodomain'] == 'Law, Governance, and Ethics']

,Macrodomain,Subject,Count
19,"Law, Governance, and Ethics",professional_law,1534
17,"Law, Governance, and Ethics",moral_scenarios,895
16,"Law, Governance, and Ethics",moral_disputes,346
18,"Law, Governance, and Ethics",philosophy,311
15,"Law, Governance, and Ethics",logical_fallacies,163
14,"Law, Governance, and Ethics",jurisprudence,108
13,"Law, Governance, and Ethics",business_ethics,100


In [ ]:
# Split dataset into train (70%) and test (30%) sets for the selected macrodomain
df_train, df_test = train_test_split(
    df_mmmlu_pt[df_mmmlu_pt['Macrodomain'] == 'Law, Governance, and Ethics'],
    test_size=0.3,                  # 30% for the test set (which leaves 70% for the train set)
    random_state=42,                # Use a fixed random state for reproducibility
    stratify=df_mmmlu_pt[df_mmmlu_pt['Macrodomain'] == 'Law, Governance, and Ethics']['Subject'] # Stratify by the 'Subject' column
)

In [ ]:
# Verify distribution in the test set
test_subject_counts = df_test.groupby(['Macrodomain', 'Subject']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)
df_test.to_csv(f"{output_path}/mmlu_test.csv", index=False) # Save
test_subject_counts

,Macrodomain,Subject,Count
6,"Law, Governance, and Ethics",professional_law,461
4,"Law, Governance, and Ethics",moral_scenarios,269
3,"Law, Governance, and Ethics",moral_disputes,104
5,"Law, Governance, and Ethics",philosophy,93
2,"Law, Governance, and Ethics",logical_fallacies,49
1,"Law, Governance, and Ethics",jurisprudence,32
0,"Law, Governance, and Ethics",business_ethics,30


In [ ]:
# Verify distribution in the train set
train_subject_counts = df_train.groupby(['Macrodomain', 'Subject']).size().reset_index(name='Count').sort_values(by='Count', ascending=False)
df_train.to_csv(f"{output_path}/mmlu_train.csv", index=False) # Save
train_subject_counts

,Macrodomain,Subject,Count
6,"Law, Governance, and Ethics",professional_law,1073
4,"Law, Governance, and Ethics",moral_scenarios,626
3,"Law, Governance, and Ethics",moral_disputes,242
5,"Law, Governance, and Ethics",philosophy,218
2,"Law, Governance, and Ethics",logical_fallacies,114
1,"Law, Governance, and Ethics",jurisprudence,76
0,"Law, Governance, and Ethics",business_ethics,70


## 3. Portuguese Wikipedia Corpus <a id="part_03"></a>

This section loads, filters, and prepares the **Portuguese Wikipedia (PT-BR)** dataset  
from [HuggingFace — *pablo-moreira/wikipedia-pt*](https://huggingface.co/datasets/pablo-moreira/wikipedia-pt).

**Main steps:**
1. Load and inspect the full Wikipedia PT-BR dataset.  
2. Filter articles using domain-specific **keyword groups** related to *Law, Governance, and Ethics*.  
3. Apply **semantic similarity filtering** with multilingual embeddings.  
4. Merge both filtering approaches into a **final curated dataset**.

> This stage builds a focused Wikipedia subset aligned with the *Law, Governance, and Ethics* macrodomain.


### 3.1 Loading the Dataset

Load and concatenate all `.parquet` files from the Wikipedia PT-BR corpus,  
creating a unified DataFrame with article titles and text content.


In [ ]:
# Load the dataset
ds = load_dataset("pablo-moreira/wikipedia-pt", "latest", split="train")

# Convert to a pandas DataFrame (optional, for convenience)
df_wiki = pd.DataFrame(ds)

print("Total articles:", len(df_wiki))
df_wiki.head()

(1857355, 3)


,id,title,text
0,220,Astronomia,Astronomia\n\nAstronomia é uma ciência natural...
1,223,América Latina,América Latina\n\nA América Latina (; ) é uma ...
2,224,Albino Forjaz de Sampaio,Albino Forjaz de Sampaio\n\nAlbino Maria Perei...
3,226,Anno Domini,Anno Domini\n\nAnno Domini (A.D.) é uma expres...
4,228,Aquiles,"Aquiles\n\nAquiles (), na mitologia grega, foi..."


### 3.2 Keyword-Based Filtering

Define **keyword groups** to identify relevant articles related to *Law, Governance, and Ethics*  
(e.g., legal terminology, governance topics, ethical concepts, and professional figures).

The keyword strategy and domain taxonomy were refined with the help of **ChatGPT 5**,  
which assisted in expanding lists with relevant Portuguese legal, political, and ethical terminology.

---

#### Prompt 1 — Domain Keywords (Law, Governance, Ethics, Business)

> “Generate comprehensive Portuguese keyword lists related to **Law, Governance, and Ethics**,  
> including legal terms, governance and policy expressions, ethical and moral vocabulary,  
> and business or corporate responsibility terminology.  
> Each list should capture stems or word roots suitable for text pattern matching.”

These lists formed the basis of `law_keywords`, `governance_keywords`, `ethics_keywords`, and `business_keywords`.

---

#### Prompt 2 — Biography Keywords

> “List common **Portuguese words or phrases** that typically appear in biographical or professional  
> descriptions of people connected to **law, governance, politics, or moral philosophy** —  
> including professions, roles, and titles (e.g., *advogado*, *filósofo*, *ativista*).  
> Focus on general descriptors likely to occur in the first sentences of Wikipedia biographies.”

This prompt generated the `bio_keywords` set used to detect professional or biographical relevance.

---

#### Prompt 3 — Named Entities (People and Thinkers)

> “Create an extensive list of **names of historical and contemporary figures** relevant to  
> *law, governance, ethics, and political or moral philosophy* in Portuguese and global contexts.  
> Include jurists, philosophers, political leaders, sociologists, activists, and thinkers  
> whose Wikipedia articles could contain valuable domain knowledge.”

This prompt produced the `name_keywords` list used for named-entity matching in the filtering stage.

---

* All three evaluated keyword sets were reviewed, refined, and integrated into the filtering pipeline  
to ensure wide coverage of the *Law, Governance, and Ethics* domain while minimizing irrelevant noise.



In [ ]:
# Domain Keywords
law_keywords = [
    "direito", "leis", "lei", "jurídic", "constituição", "justiça", "crime",
    "criminal", "penal", "civil", "advogad", "tribunal", "senten", "norma",
    "litígio", "processo", "recurso", "constitucional", "supremo", "judicial",
    "magistrad", "jurisprud", "procurador", "ministerio público"
]

governance_keywords = [
    "governo", "governança", "estado", "poder público", "administra",
    "política pública", "políticas públicas", "soberania", "cidadania",
    "direitos humanos", "parlamento", "congresso", "senado", "câmara",
    "prefeit", "corrupção", "transparência", "accountability", "eleição",
    "eleições", "partido político", "democracia", "constitucionalismo"
]

ethics_keywords = [
    "ética", "moral", "deontolog", "utilitar", "kant", "virtude", "faláci",
    "falácia", "filosofia moral", "dilema", "bioética", "moralidade",
    "responsabilidade", "justiça social", "igualdade", "liberdade",
    "honestidade", "corrupção ética"
]

business_keywords = [
    "governança corporativa", "compliance", "responsabilidade social",
    "ética empresarial", "conduta profissional", "transparência corporativa",
    "corrupção corporativa", "responsabilidade socioambiental"
]

# Biography Keywords
bio_keywords = [
    # Legal professions
    "jurista", "advogado", "advogada", "magistrado", "magistrada",
    "procurador", "procuradora", "ministro do supremo", "juiz", "juíza",
    
    # Political/governance roles
    "político", "política", "estadista", "governante", "presidente", 
    "parlamentar", "deputado", "senador", "prefeito", "ministro", "governador",
    
    # Academic / philosophical
    "filósofo", "filósofa", "pensador", "pensadora", "teórico", "teórica",
    "professor de filosofia", "moralista", "teólogo", "intelectual público",
    
    # Ethics / activism
    "ativista", "defensor dos direitos humanos", "reformador", "humanista"
]

# Named Entity Keywords
name_keywords = [
    # --- Classic jurists and legal thinkers ---
    "Rui Barbosa", "Clóvis Beviláqua", "Tobias Barreto", "Pontes de Miranda",
    "Miguel Reale", "Miguel Reale Júnior", "Paulo Bonavides", "Celso Antônio Bandeira de Mello",
    "José Afonso da Silva", "Fábio Konder Comparato", "Raymundo Faoro", "Carlos Maximiliano",
    "Pimenta Bueno", "Castro Nunes", "Teixeira de Freitas", "Nelson Hungria", "Vicente Ráo",
    "José Levi Mello do Amaral", "Luis Roberto Barroso", "Gilmar Mendes", "Joaquim Barbosa",
    "Cármen Lúcia", "Alexandre de Moraes", "Ayres Britto", "Sepúlveda Pertence", "Eros Grau",
    "Marco Aurélio Mello", "Sérgio Moro", "Maria Berenice Dias", "Luís Felipe Salomão",

    # --- Political and constitutional figures ---
    "Ulysses Guimarães", "Tancredo Neves", "José Sarney", "Itamar Franco",
    "Fernando Henrique Cardoso", "Lula", "Luiz Inácio Lula da Silva", "Dilma Rousseff",
    "Michel Temer", "Jair Bolsonaro", "Getúlio Vargas", "Juscelino Kubitschek",
    "João Goulart", "Castelo Branco", "Costa e Silva", "Ernesto Geisel", "Figueiredo",
    "José Bonifácio de Andrada e Silva", "Afonso Arinos de Melo Franco", "Teotônio Vilela",
    "Leonel Brizola", "Darcy Ribeiro", "Celso Furtado", "Celso Lafer", "Mario Covas",
    "Eduardo Campos", "Marina Silva", "Ciro Gomes", "Fernando Haddad", "Sérgio Cabral",
    "Antonio Carlos Magalhães", "José Serra", "Aécio Neves", "Geraldo Alckmin",

    # --- Philosophers, sociologists, educators, and thinkers ---
    "Paulo Freire", "Florestan Fernandes", "Roberto Mangabeira Unger", "Marilena Chauí",
    "Renato Janine Ribeiro", "Leandro Konder", "Luiz Felipe Pondé", "Olavo de Carvalho",
    "Mario Sérgio Cortella", "Clóvis de Barros Filho", "Vladimir Safatle", "Emir Sader",
    "Milton Santos", "Gilberto Freyre", "Sérgio Buarque de Holanda", "Caio Prado Júnior",
    "Darcy Ribeiro", "Euclides da Cunha", "José Guilherme Merquior", "Antonio Candido",
    "Nelson Rodrigues", "Nina Rodrigues", "Gilberto Amado", "Afrânio Coutinho",

    # --- Activists, journalists, and human rights defenders ---
    "Maria da Penha", "Marielle Franco", "Zilda Arns", "Herbert de Souza", "Betinho",
    "Dom Hélder Câmara", "Chico Mendes", "Carlos Drummond de Andrade", "Clarice Lispector",
    "Ariano Suassuna", "Aldaíza Sposati", "Nise da Silveira", "Heleieth Saffioti",
    "Lélia Gonzalez", "Djamila Ribeiro", "Carolina Maria de Jesus", "Sueli Carneiro",
    "Abdias do Nascimento", "Milton Santos", "Leonardo Boff", "Frei Betto",
    "Rubem Alves", "Henfil", "Millôr Fernandes", "Ferreira Gullar", "Patrícia Campos Mello"

    # --- Classical philosophy ---
    "Sócrates", "Platão", "Aristóteles", "Epicuro", "Cícero", "Sêneca", "Agostinho de Hipona",
    "Tomás de Aquino", "Maquiavel", "Francis Bacon", "René Descartes", "Baruch Spinoza",
    "David Hume", "John Locke", "Thomas Hobbes", "Jean-Jacques Rousseau",
    "Montesquieu", "Voltaire", "Immanuel Kant", "Georg Wilhelm Friedrich Hegel",
    "Karl Marx", "Friedrich Engels", "John Stuart Mill", "Jeremy Bentham",
    "Alexis de Tocqueville", "Giambattista Vico", "Edmund Burke", "Thomas Paine",

    # --- Modern philosophy & ethics ---
    "Friedrich Nietzsche", "Arthur Schopenhauer", "Auguste Comte", "Emile Durkheim",
    "Max Weber", "Karl Popper", "Ludwig Wittgenstein", "Hannah Arendt",
    "Michel Foucault", "Jürgen Habermas", "John Rawls", "Robert Nozick",
    "Alasdair MacIntyre", "Peter Singer", "Amartya Sen", "Martha Nussbaum",
    "Simone de Beauvoir", "Jean-Paul Sartre", "Albert Camus", "Paul Ricoeur",
    "Emmanuel Levinas", "Charles Taylor", "Isaiah Berlin", "Erich Fromm",

    # --- Legal theory & jurisprudence ---
    "Hans Kelsen", "Herbert Hart", "Ronald Dworkin", "Lon Fuller", "Carl Schmitt",
    "Cesare Beccaria", "Gustav Radbruch", "Michel Villey", "Roscoe Pound",
    "Oliver Wendell Holmes", "Benjamin Cardozo", "John Austin", "Jeremy Bentham",
    "John Finnis", "Joseph Raz", "Richard Posner", "Cass Sunstein",

    # --- Political theory, governance, economics ---
    "John Maynard Keynes", "Friedrich Hayek", "Karl Polanyi", "Antonio Gramsci",
    "Ernesto Laclau", "Chantal Mouffe", "Robert Dahl", "Giovanni Sartori",
    "Max Weber", "David Easton", "John Dewey", "Niccolò Machiavelli", "Raymond Aron",
    "Hannah Pitkin", "Elinor Ostrom", "Amartya Sen", "Douglass North",

    # --- Moral and applied ethics ---
    "Peter Singer", "Philippa Foot", "Judith Jarvis Thomson", "Elizabeth Anscombe",
    "Bernard Williams", "Thomas Nagel", "Christine Korsgaard", "Charles Taylor",
    "Susan Wolf", "Michael Sandel", "Onora O'Neill", "Iris Murdoch",

    # --- Human rights and social thought ---
    "Eleanor Roosevelt", "Nelson Mandela", "Martin Luther King", "Mahatma Gandhi",
    "Desmond Tutu", "Vaclav Havel", "Aung San Suu Kyi", "Malala Yousafzai",
    "Noam Chomsky", "Cornel West", "bell hooks", "Angela Davis", "Judith Butler"
]

This cell defines and compiles **regular expression patterns** used to identify relevant Wikipedia articles.

It first wraps keyword lists (law, governance, ethics, business, biography, and names) into **compiled regex objects**  
for efficient text search. These patterns detect domain-specific terms, named entities, and biography markers.  

A separate `exclude_re` pattern filters out irrelevant topics (e.g., sports, music).  
Finally, all domain keywords are merged into a single `domain_pattern` for global keyword counting.

In [4]:
def compile_pattern(words):
    return re.compile(r"\b(?:" + "|".join(map(re.escape, words)) + r")\b", flags=re.IGNORECASE)

# Base keyword groups
law_re              = compile_pattern(law_keywords)
governance_re       = compile_pattern(governance_keywords)
ethics_re           = compile_pattern(ethics_keywords)
business_re         = compile_pattern(business_keywords)
bio_re              = compile_pattern(bio_keywords)
name_re             = re.compile("|".join(map(re.escape, name_keywords)), flags=re.IGNORECASE)
exclude_re          = re.compile(r"(?i)\b(lista|campeonato|futebol|álbum|música|temporada|condado|jogos?)\b")

# Combined domain pattern for efficient counting
all_domain_keywords = law_keywords + governance_keywords + ethics_keywords + business_keywords
domain_pattern      = compile_pattern(all_domain_keywords)

This cell implements the core **filtering logic** applied to each Wikipedia chunk.

The function `process_chunk()` scans article titles and texts to create boolean masks  
based on multiple inclusion and exclusion rules:

1. **Named-entity rule** — selects articles mentioning known domain figures.  
2. **Biography rule** — detects professional or biographical indicators in the introduction.  
3. **Domain keyword rule** — keeps articles exceeding a keyword threshold.  
4. **Exclusion rule** — removes entertainment or unrelated topics.

It returns masks and statistics summarizing how many articles matched each rule.

In [5]:
def process_chunk(chunk_df, threshold=5):
    """
    Process a chunk of the dataframe and return boolean mask for inclusion.
    
    Args:
        chunk_df: DataFrame chunk with 'title' and 'text' columns
        threshold: Minimum domain keyword count for inclusion
    
    Returns:
        numpy array of boolean values indicating which rows to include
    """
    titles = chunk_df["title"].fillna("").astype(str)
    texts = chunk_df["text"].fillna("").astype(str)
    
    # Rule 1: Named-entity rule (check title or multiple mentions in text)
    mask_name_title = titles.str.contains(name_re, na=False, regex=True)
    
    # For text, count occurrences and select if >= 4 mentions
    name_counts = texts.str.count(name_re)
    mask_name_text = name_counts >= 4
    mask_name = mask_name_title | mask_name_text
    
    # Rule 2: Biography rule (profession keyword in first 200 chars)
    intro_snippet = texts.str.slice(0, 200)
    mask_bio = intro_snippet.str.contains(bio_re, na=False, regex=True)
    
    # Rule 3: Domain keyword scoring (count all domain keywords)
    domain_counts = texts.str.count(domain_pattern)
    mask_domain = domain_counts >= threshold
    
    # Combine inclusion rules
    mask_include = mask_name | mask_bio | mask_domain
    
    # Apply exclusion filter
    mask_exclude = titles.str.contains(exclude_re, na=False, regex=True)
    
    # Final mask
    final_mask = mask_include & ~mask_exclude

    return (final_mask.values, 
            mask_exclude.sum(), 
            mask_name.sum(), 
            mask_bio.sum(), 
            mask_domain.sum())


This cell defines `filter_wikipedia_parallel()`, which executes the filtering pipeline in **parallel threads**  
for scalability on large Wikipedia datasets.

It splits the DataFrame into chunks, processes each using `process_chunk()`,  
and aggregates results while tracking detailed metrics — including total matches, exclusions,  
and overall reduction percentage.

The function prints a summary report and returns both the filtered dataset and a statistics dictionary.

In [6]:
def filter_wikipedia_parallel(df, threshold=5, n_workers=None, chunk_size=10000):
    """
    Filter Wikipedia dataframe in parallel using ThreadPoolExecutor.
    
    Args:
        df: Input DataFrame with 'title' and 'text' columns
        threshold: Minimum domain keyword count
        n_workers: Number of worker threads (default: CPU count)
        chunk_size: Number of rows per chunk
    
    Returns:
        tuple: (filtered_df, stats_dict)
    """
    print(f"Processing {len(df):,} rows with {n_workers or 'auto'} workers...")
    
    # Split dataframe into chunks
    chunks = [df.iloc[i:i + chunk_size].copy() for i in range(0, len(df), chunk_size)]
    n_chunks = len(chunks)
    print(f"Split into {n_chunks} chunks of ~{chunk_size:,} rows each")
    
    # Process chunks in parallel
    results = []
    process_fn = partial(process_chunk, threshold=threshold)
    
    # Initialize statistics accumulators
    total_excluded = 0
    total_name = 0
    total_bio = 0
    total_domain = 0
    
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        # Submit all chunks
        future_to_idx = {executor.submit(process_fn, chunk): i 
                        for i, chunk in enumerate(chunks)}
        
        # Collect results as they complete
        for i, future in enumerate(tqdm(as_completed(future_to_idx), total=n_chunks, desc="Processing chunks")):
            chunk_idx = future_to_idx[future]
            try:
                mask, excluded, name_cnt, bio_cnt, domain_cnt = future.result()
                results.append((chunk_idx, mask))
                
                # Accumulate statistics
                total_excluded += excluded
                total_name += name_cnt
                total_bio += bio_cnt
                total_domain += domain_cnt
                
                print(f"Chunk {i}/{n_chunks} (idx {chunk_idx}): "
                      f"{mask.sum():,} selected | "
                      f"name={name_cnt} bio={bio_cnt} domain={domain_cnt} excluded={excluded}")
            except Exception as e:
                print(f"Error processing chunk {chunk_idx}: {e}")
                raise
    
    # Sort results by original chunk order and concatenate masks
    results.sort(key=lambda x: x[0])
    full_mask = np.concatenate([mask for _, mask in results])
    
    # Apply mask to original dataframe
    df_filtered = df[full_mask].copy()
    
    # Calculate overlap statistics (rules can overlap)
    total_selected = len(df_filtered)
    
    # Compile statistics
    stats = {
        "total_rows": len(df),
        "selected_rows": total_selected,
        "reduction_pct": (1 - total_selected / len(df)) * 100,
        "excluded_by_filter": total_excluded,
        "matched_name_rule": total_name,
        "matched_bio_rule": total_bio,
        "matched_domain_rule": total_domain,
        "threshold_used": threshold
    }
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"FILTERING COMPLETE")
    print(f"{'='*60}")
    print(f"Original rows:        {stats['total_rows']:>10,}")
    print(f"Filtered rows:        {stats['selected_rows']:>10,}")
    print(f"Reduction:            {stats['reduction_pct']:>10.1f}%")
    print(f"\nRule Matches (with overlaps):")
    print(f"  Named entities:     {stats['matched_name_rule']:>10,}")
    print(f"  Biography keywords: {stats['matched_bio_rule']:>10,}")
    print(f"  Domain keywords:    {stats['matched_domain_rule']:>10,}")
    print(f"\nExcluded (sport/entertainment): {stats['excluded_by_filter']:>6,}")
    print(f"Domain threshold used:          {stats['threshold_used']:>6}")
    print(f"{'='*60}\n")
    
    return df_filtered, stats

This cell executes the full **parallel filtering process** and exports the resulting dataset.

It calls `filter_wikipedia_parallel()` with the configured parameters (e.g., threshold, number of workers),  
then computes an additional metric: the **average number of rule matches per article**.  

Finally, it saves the filtered output to `wiki_keyword.csv`, providing the curated Wikipedia subset  
aligned with the *Law, Governance, and Ethics* domain.

> ⚠️ **Warning:** The next Python cell involves **high-performance computing (HPC)**.  
> Execution requires a **dedicated or cloud machine with multiple cores**, not a standard desktop or notebook.  
> Runtime and cell outputs are reported below.


In [ ]:
if __name__ == "__main__":
    # Filter with parallel processing
    df_filtered, stats = filter_wikipedia_parallel(
        df_wiki, 
        threshold=5,
        n_workers=10,  # or None for auto
        chunk_size=20000
    )
    
    # Access statistics
    print("Additional Analysis:")
    print(f"Average rules matched per article: "
          f"{(stats['matched_name_rule'] + stats['matched_bio_rule'] + stats['matched_domain_rule']) / stats['selected_rows']:.2f}")
    
    # Save results
    df_filtered.to_csv(f"{output_path}/wiki_keyword.csv", index=False)

df_filtered

Processing 1,857,355 rows with 10 workers...
Split into 93 chunks of ~20,000 rows each


Processing chunks:   0%|          | 0/93 [00:00<?, ?it/s]/tmp/ipykernel_914/4185289174.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_exclude = titles.str.contains(exclude_re, na=False, regex=True)
Processing chunks:   1%|          | 1/93 [06:22<9:47:07, 382.90s/it]

Chunk 0/93 (idx 6): 958 selected | name=129 bio=554 domain=626 excluded=435


/tmp/ipykernel_914/4185289174.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_exclude = titles.str.contains(exclude_re, na=False, regex=True)
Processing chunks:   2%|▏         | 2/93 [07:43<5:11:00, 205.06s/it]

Chunk 1/93 (idx 9): 932 selected | name=112 bio=416 domain=592 excluded=495


...

/tmp/ipykernel_914/4185289174.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_exclude = titles.str.contains(exclude_re, na=False, regex=True)
Processing chunks:  99%|█████████▉| 92/93 [43:19<00:17, 17.45s/it]

Chunk 91/93 (idx 90): 2,045 selected | name=157 bio=1087 domain=1297 excluded=1510


/tmp/ipykernel_914/4185289174.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_exclude = titles.str.contains(exclude_re, na=False, regex=True)
Processing chunks: 100%|██████████| 93/93 [43:21<00:00, 27.97s/it]


Chunk 92/93 (idx 91): 2,233 selected | name=134 bio=1295 domain=1318 excluded=1427

FILTERING COMPLETE
Original rows:         1,857,355
Filtered rows:           108,150
Reduction:                  94.2%

Rule Matches (with overlaps):
  Named entities:         10,006
  Biography keywords:     58,438
  Domain keywords:        63,646

Excluded (sport/entertainment): 87,705
Domain threshold used:               5

Additional Analysis:
Average rules matched per article: 1.22


,id,title,text
0,220,Astronomia,Astronomia\n\nAstronomia é uma ciência natural...
1,223,América Latina,América Latina\n\nA América Latina (; ) é uma ...
2,224,Albino Forjaz de Sampaio,Albino Forjaz de Sampaio\n\nAlbino Maria Perei...
3,226,Anno Domini,Anno Domini\n\nAnno Domini (A.D.) é uma expres...
5,229,Anarcocapitalismo,Anarcocapitalismo\n\nAnarcocapitalismo (também...
...,...,...,...
1857297,7219343,Azerbaijão Ocidental (conceito político),Azerbaijão Ocidental (conceito político)\n\nAz...
1857305,7219375,Ferdinand Berthier,"Ferdinand Berthier\n\n \nFerdinand Berthier (,..."
1857325,7219482,Camille Dimmer,"Camille Dimmer\n\nCamille Dimmer (Clervaux, 20..."
1857335,7219529,Supremo Tribunal Federal da Suíça,Supremo Tribunal Federal da Suíça\n\nO Supremo...


### 3.3 Semantic Filtering with Embeddings

This step extends the filtering process beyond explicit keywords by using **semantic similarity**.  
Articles are selected based on how closely their textual content aligns conceptually with  
*Law, Governance, and Ethics* through **multilingual sentence embeddings**.

A **SentenceTransformer model** (`paraphrase-multilingual-MiniLM-L12-v2`) is used to encode both  
Wikipedia article titles and domain-representative **seed sentences**, which serve as semantic anchors.

The design of the seed sentences and conceptual scope was developed with the assistance of **ChatGPT 5**,  
which helped formulate concise and representative textual prompts describing the domain themes  
(*law, governance, ethics, and logic*).

---

#### Prompt used with ChatGPT 5

> “Create a set of short Portuguese sentences that represent the main themes of the  
> *Law, Governance, and Ethics* macrodomain.  
> Include references to legal systems, justice, human rights, public governance,  
> moral philosophy, and logical reasoning.  
> The sentences should capture semantic meaning suitable for use as embedding seeds.”

These seed sentences were encoded and compared against article titles using **cosine similarity**.  
Articles whose maximum similarity exceeded the threshold (`0.48`) were retained as semantically relevant.

---

* This semantic approach complements keyword filtering by capturing **contextual and conceptual matches**  
beyond explicit terminology, improving the coverage and precision of the final Wikipedia subset.


In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Define macrodomain seed sentences
seed_texts = [
    # --- Legal Systems & Jurisprudence ---
    "direito civil, penal e constitucional, estudo das leis e da justiça",
    "tribunais, constituição, legislação e aplicação do direito",
    "jurisprudência, processos judiciais e princípios jurídicos",
    "direitos humanos, igualdade perante a lei e cidadania",

    # --- Governance & Public Institutions ---
    "governança democrática, estado de direito e políticas públicas",
    "poder legislativo, executivo e judiciário na administração pública",
    "ética pública, responsabilidade política e combate à corrupção",
    "relações internacionais sob a ótica jurídica e diplomática",

    # --- Moral Philosophy & Ethics ---
    "filosofia moral, ética normativa e dilemas éticos contemporâneos",
    "teorias éticas como utilitarismo, deontologia e virtudes morais",
    "bioética, justiça social e responsabilidade individual e coletiva",
    "ética empresarial e governança corporativa responsável",

    # --- Logic & Critical Thinking ---
    "falácias lógicas, raciocínio dedutivo e análise de argumentos",
    "pensamento crítico e filosofia da razão aplicada à ética e ao direito",
]

seed_emb = model.encode(seed_texts, convert_to_tensor=True)

> ⚠️ **Warning:** The next Python cell involves **high-performance computing (HPC)**.  
> Execution requires a **dedicated or cloud machine with multiple cores**, not a standard desktop or notebook.  


In [ ]:
# Compute cosine similarity with article titles
# titles_emb = model.encode(df_wiki["title"].tolist(), convert_to_tensor=True, batch_size=256)
# torch.save(titles_emb, f"{output_path}/wiki_titles_emb.pt")

# ====== OR ========

# Import saved embeddings
titles_emb = torch.load(f"{output_path}/wiki_titles_emb.pt")

This cell applies the similarity threshold to create a mask of selected articles  
and extracts those most semantically related to the defined domain.  

It prints the total number of selected articles and analyzes the most frequent tokens  
from their titles, providing a quick lexical overview of the filtered subset.  

The resulting `df_semantic` DataFrame contains Wikipedia articles that were  
**selected purely by semantic similarity** rather than explicit keyword matches.

In [12]:
# Compute all cosine similarities (matrix of shape [N_titles, N_seeds])
sims_matrix = util.cos_sim(titles_emb, seed_emb)

# Take the maximum similarity per title (best match with any seed)
sims, _ = torch.max(sims_matrix, dim=1)

# Select threshold (you can tune between 0.35 and 0.45)
threshold = 0.48
mask_semantic = sims > threshold

df_semantic = df_wiki[mask_semantic.cpu().numpy()].copy()

print(f"Selected {len(df_semantic):,} articles above similarity {threshold}")

# Check top tokens
words = re.findall(r'\b[a-zà-úA-ZÀ-Ú]{4,}\b', " ".join(df_semantic['title'].tolist()).lower())
print("Most common terms in titles:", Counter(words).most_common(30))

df_semantic

Selected 26,428 articles above similarity 0.48
Most common terms in titles: [('lista', 1227), ('governo', 863), ('legislatura', 652), ('tratado', 643), ('direito', 627), ('estado', 626), ('deputados', 604), ('conselho', 603), ('nações', 536), ('política', 535), ('república', 511), ('estaduais', 498), ('unidas', 492), ('social', 490), ('justiça', 489), ('estados', 447), ('direitos', 437), ('tribunal', 424), ('internacional', 423), ('nacional', 393), ('segurança', 389), ('federal', 386), ('para', 381), ('partido', 378), ('relações', 360), ('unidos', 347), ('geral', 336), ('constituição', 331), ('público', 331), ('resolução', 320)]


,id,title,text
105,367,Administração,Administração\n\nA é a ciência social que estu...
122,386,Análise matemática,Análise matemática\n\nAnálise é o ramo da mate...
144,416,Hino da Proclamação da República,Hino da Proclamação da República\n\nO Hino à P...
165,460,Behaviorismo radical,Behaviorismo radical\n\nBehaviorismo radical é...
166,461,Behaviorismo metodológico,Behaviorismo metodológico\n\nO Behaviorismo Me...
...,...,...,...
1856973,7218014,Bandeira dos Direitos Humanos,Bandeira dos Direitos Humanos\n\nA Bandeira do...
1857015,7218272,Serviço Diplomático da Letônia no exílio,Serviço Diplomático da Letônia no exílio\n\nO ...
1857152,7218800,Instituto de Estudos Políticos de Grenoble,Instituto de Estudos Políticos de Grenoble\n\n...
1857335,7219529,Supremo Tribunal Federal da Suíça,Supremo Tribunal Federal da Suíça\n\nO Supremo...


### 3.4 Final Dataset

In this final step, both filtering strategies — **keyword-based** and **semantic embedding-based** —  
are combined to build the consolidated Wikipedia dataset for the *Law, Governance, and Ethics* macrodomain.

Articles selected by either approach are merged, and **duplicates are removed** to ensure data integrity.  
The resulting dataset captures both **explicit textual relevance** (via keywords and named entities)  
and **implicit semantic alignment** (via embeddings).

The merged DataFrame is exported as `wiki_final.csv`, and basic lexical statistics are computed  
to inspect the most frequent words appearing in article titles.


In [ ]:
# Join both approaches - name/bio/domain keywords and semantic similarity
df_final = pd.concat([df_filtered, df_semantic]).drop_duplicates().reset_index(drop=True)

# Count duplicates
duplicates = pd.concat([df_filtered, df_semantic]).duplicated(keep=False).sum()
print(f"✅ Joined datasets with {duplicates:,} duplicates removed.")

# Save final DataFrame
df_final.to_csv(f"{output_path}/wiki_final.csv", index=False)
df_final

✅ Joined datasets with 12,314 duplicates removed.


,id,title,text
0,220,Astronomia,Astronomia\n\nAstronomia é uma ciência natural...
1,223,América Latina,América Latina\n\nA América Latina (; ) é uma ...
2,224,Albino Forjaz de Sampaio,Albino Forjaz de Sampaio\n\nAlbino Maria Perei...
3,226,Anno Domini,Anno Domini\n\nAnno Domini (A.D.) é uma expres...
4,229,Anarcocapitalismo,Anarcocapitalismo\n\nAnarcocapitalismo (também...
...,...,...,...
128416,7215469,O Condenado (1921),O Condenado (1921)\n\nO Condenado é um filme m...
128417,7216611,Especialista em regulação,Especialista em regulação\n\n\n\n
128418,7216852,Conselho Nacional de Combate à Discriminação,Conselho Nacional de Combate à Discriminação\n...
128419,7217114,Ovo mundial,Ovo mundial\n\n\n\n
